# Part 1: training the model

In [3]:
%load_ext autoreload
%autoreload 2
%reset -f
from custom_imports import *
from mri_project.binary_to_multi_label import get_contours_features_and_colors, binary_to_multilabel
data_root = "/data/sesek2/data/"
files = glob.glob(f'{data_root}masked/*png')
masked_images_   = joblib.load(f'{data_root}cleaned_muscles.pkl')
image_names = joblib.load(f"{data_root}image_names.pkl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
clssfd = dict(zip(files, map(cv2.imread, files)))
X = []
Y = []
for img0 in clssfd.values():
    x, y = get_contours_features_and_colors(img0)
    X.append(x)
    Y.append(y)
w = X[0].shape[1]
X = np.concatenate(X).reshape(-1, w)
Y_ = np.concatenate(Y).reshape(-1)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
#encoder = MultiLabelBinarizer()
#Y = encoder.fit_transform(Y_)
Y = Y_
Xtr, Xts, Ytr, Yts = train_test_split(X, Y, random_state=15, test_size=.6)

In [7]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, n_jobs=50, random_state=16)

In [8]:
model.fit(Xtr, Ytr)

RandomForestClassifier(n_estimators=200, n_jobs=50, random_state=16)

In [9]:
joblib.dump(model, "../data/models/random_forest/binary_to_multiclass_v02.model")

['../data/models/random_forest/binary_to_multiclass_v02.model']

In [10]:
model.score(Xts, Yts), model.score(Xtr, Ytr)

(0.9943502824858758, 1.0)

In [12]:
#encoder.inverse_transform(model.predict(x))
model.predict(x)

array([210, 160, 200, 170, 220, 150, 230, 240, 250, 180, 190], dtype=uint8)

In [13]:
muscle_colors = dict([x.split(',') for x in """Left banana  ,250
Right banana  ,240
Left central  ,230
Right central  ,220
Bottom left  ,210
Bottom right  ,200
Top left  ,190
Top right  ,180
Btw bottom and center left  ,170
Btw bottom and center right  ,160
Center  ,150
Left half banana  ,140
Right half banana  ,130""".split('\n')])
reverse_muscle_colors = {int(v): k for k, v in muscle_colors.items()}
print(muscle_colors)
print(reverse_muscle_colors)

{'Left banana  ': '250', 'Right banana  ': '240', 'Left central  ': '230', 'Right central  ': '220', 'Bottom left  ': '210', 'Bottom right  ': '200', 'Top left  ': '190', 'Top right  ': '180', 'Btw bottom and center left  ': '170', 'Btw bottom and center right  ': '160', 'Center  ': '150', 'Left half banana  ': '140', 'Right half banana  ': '130'}
{250: 'Left banana  ', 240: 'Right banana  ', 230: 'Left central  ', 220: 'Right central  ', 210: 'Bottom left  ', 200: 'Bottom right  ', 190: 'Top left  ', 180: 'Top right  ', 170: 'Btw bottom and center left  ', 160: 'Btw bottom and center right  ', 150: 'Center  ', 140: 'Left half banana  ', 130: 'Right half banana  '}


In [12]:
n = np.random.randint(len(masked_images_))
multi_label_images = {}

for image_name, img in zip(image_names, masked_images_):
    img = img-1
    imt = binary_to_multilabel(img, model) 
    multi_label_images[image_name] = imt

In [15]:
#joblib.dump(multi_label_images, "data/multi_label_images.dict")

['data/multi_label_images.dict']

In [11]:
import pandas as pd

In [19]:
tmp_ = pd.DataFrame([['B', 10000, 25000], ['D', 8000, 20000]], columns=['Program', 'Reach', 'impression'])

In [22]:
tmp_.set_index('Program').unstack().to_frame()

0
           Program       
Reach      B        10000
           D         8000
impression B        25000
           D        20000